<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250625/%E3%83%80%E3%82%A4%E3%82%A8%E3%83%83%E3%83%88%E5%95%8F%E9%A1%8C%E3%81%AE%E6%BC%94%E7%BF%92%E5%95%8F%E9%A1%8C%E3%81%AE%E8%A7%A3%E8%AA%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 準備

必要なライブラリｎインストールとインポート

In [1]:
%%capture
# 最適化ライブラリ python-mip のインストール
!pip install mip

# データフレーム操作のための pandas のインポート
import pandas as pd
# 最適化ライブラリ python-mip のインポート
import mip

## 元の問題設定のデータ準備

各演習問題で使う元のダイエット問題のデータを準備


In [3]:
# 食品データの作成
data = {
    '食品名': ['牛乳', '卵', 'パン', 'チーズ', 'オレンジ'],
    '単位': ['100ml', '1個', '1枚', '20g', '1個'],
    'コスト': [20, 30, 40, 50, 60],
    'カロリー': [60, 80, 120, 80, 50],
    'タンパク質': [3, 6, 4, 5, 1],
    'カルシウム': [110, 30, 20, 140, 40]
}
food_df = pd.DataFrame(data)
food_df = food_df.set_index('食品名')

# 栄養基準データの作成
min_requirements = {
    'カロリー': 500,
    'タンパク質': 25,
    'カルシウム': 400
}

# MIPで扱いやすい形式に変換
foods = food_df.index.tolist()
costs = food_df['コスト'].to_dict()
nutrients = ['カロリー', 'タンパク質', 'カルシウム']

# 各食品の栄養価を {食品名: {栄養素名: 値, ...}, ...} の形式に変換
nutrient_values = food_df[nutrients].to_dict('index')


---

## 演習問題 1

### 1. 問題の確認
「牛乳」アレルギーのため、牛乳を摂取できない場合の最小コストを解答せよ。

### 2. 数理モデルの定式化
この問題を解くには、元のモデルに「牛乳の摂取量を0にする」という制約を追加する。

$$x_{牛乳} = 0$$

### 3. Python (MIP) による実装
以下のようにすることで、変数を定義する際に上限 (ub) を0に設定する。
```
x1['牛乳'].ub = 0.0
```

In [4]:
# --- 演習問題1 ---

# --- モデルの作成 ---
m1 = mip.Model(name="Diet_Ex1", sense=mip.MINIMIZE)

# --- 変数の定義 ---
# 牛乳の摂取量の上限を0に設定する
x1 = {j: m1.add_var(name=f"x_{j}", var_type=mip.CONTINUOUS, lb=0.0) for j in foods}
x1['牛乳'].ub = 0.0 # 牛乳の上限を0に設定

# --- 目的関数の設定 ---
m1.objective = mip.xsum(costs[j] * x1[j] for j in foods)

# --- 制約条件の追加 ---
for i in nutrients:
    m1.add_constr(
        mip.xsum(nutrient_values[j][i] * x1[j] for j in foods) >= min_requirements[i],
        name=f"Nutrient_{i}"
    )

# --- 求解 ---
status1 = m1.optimize()

# --- 結果と考察 ---
print("--- 演習問題1 解答 ---")
if status1 == mip.OptimizationStatus.OPTIMAL:
    print(f"最小コスト: {m1.objective_value:.2f}円")
    print("\n[各食品の最適摂取量]")
    for j in foods:
        if x1[j].x > 1e-6:
            print(f"{j}: {x1[j].x:.2f} {food_df.loc[j, '単位']}")
else:
    print("最適解が見つかりませんでした。")

--- 演習問題1 解答 ---
最小コスト: 225.22円

[各食品の最適摂取量]
卵: 0.83 1個
パン: 2.02 1枚
チーズ: 2.39 20g


---

## 演習問題 2

### 1. 問題の確認
元の問題でタンパク質の最小必要摂取量が30gに増加した場合の最小コストを解答せよ。

### 2. 数理モデルの定式化
タンパク質に関する最小摂取量の定数 $b_{タンパク質}^{min}$ を 30 に変更する。

変更後の制約:
$$\sum_{j \in F} a_{タンパク質, j} x_j \ge 30$$

他の制約条件および目的関数に変更はない。

### 3. Python (MIP) による実装
`min_requirements` ディクショナリ内のタンパク質の値を30に変更し、
再度最適化計算を行う。

In [5]:
# --- 演習問題2：データ準備 ---
# タンパク質の最小摂取量を変更
min_requirements_ex2 = min_requirements.copy()
min_requirements_ex2['タンパク質'] = 30

# --- モデルの作成 ---
m2 = mip.Model(name="Diet_Ex2", sense=mip.MINIMIZE)

# --- 変数の定義 ---
x2 = {j: m2.add_var(name=f"x_{j}", var_type=mip.CONTINUOUS, lb=0.0) for j in foods}

# --- 目的関数の設定 ---
m2.objective = mip.xsum(costs[j] * x2[j] for j in foods)

# --- 制約条件の追加 ---
for i in nutrients:
    m2.add_constr(
        mip.xsum(nutrient_values[j][i] * x2[j] for j in foods) >= min_requirements_ex2[i],
        name=f"Nutrient_{i}"
    )

# --- 求解 ---
status2 = m2.optimize()

# --- 結果と考察 ---
print("--- 演習問題2 解答 ---")
if status2 == mip.OptimizationStatus.OPTIMAL:
    print(f"最小コスト: {m2.objective_value:.2f}円")
    print("\n[各食品の最適摂取量]")
    for j in foods:
        if x2[j].x > 1e-6:
            print(f"{j}: {x2[j].x:.2f} {food_df.loc[j, '単位']}")
else:
    print("最適解が見つかりませんでした。")

--- 演習問題2 解答 ---
最小コスト: 175.00円

[各食品の最適摂取量]
牛乳: 5.00 100ml
卵: 2.50 1個


---

## 演習問題 3

### 1. 問題の確認
元の問題に対して、新たに「ヨーグルト」が利用可能になった。ヨーグルトは100gあたりコスト25円、カロリー 90kcal、タンパク質 4g、カルシウム120mgである。ヨーグルトを追加した場合の最小コストを解答せよ。

### 2. 数理モデルの定式化
この問題では、食品の集合 $F$ に「ヨーグルト」が追加される。

$$F' = F \cup \{ヨーグルト\}$$

これに伴い、決定変数 $x_{ヨーグルト}$ が追加され、目的関数と制約条件の総和にこの新しい変数が含まれることになる。
パラメータもヨーグルトのものが追加される。
* $c_{ヨーグルト} = 25$
* $a_{カロリー, ヨーグルト} = 90$
* $a_{タンパク質, ヨーグルト} = 4$
* $a_{カルシウム, ヨーグルト} = 120$

### 3. Python (MIP) による実装
食品データ `foods`, `costs`, `nutrient_values` に「ヨーグルト」の情報を追加し、モデルを再構築して最適化を行う。

In [6]:
# --- 演習問題3：データ準備 ---
# 元のデータにヨーグルトを追加
foods_ex3 = foods + ['ヨーグルト']
costs_ex3 = costs.copy()
costs_ex3['ヨーグルト'] = 25 #辞書型の変数に要素を追加

nutrient_values_ex3 = nutrient_values.copy()
nutrient_values_ex3['ヨーグルト'] = {'カロリー': 90, 'タンパク質': 4, 'カルシウム': 120}

# 単位を管理するデータも更新
food_units_ex3 = food_df['単位'].to_dict()
food_units_ex3['ヨーグルト'] = '100g'


# --- モデルの作成 ---
m3 = mip.Model(name="Diet_Ex3", sense=mip.MINIMIZE)

# --- 変数の定義 ---
x3 = {j: m3.add_var(name=f"x_{j}", var_type=mip.CONTINUOUS, lb=0.0) for j in foods_ex3}

# --- 目的関数の設定 ---
m3.objective = mip.xsum(costs_ex3[j] * x3[j] for j in foods_ex3)

# --- 制約条件の追加 ---
for i in nutrients:
    m3.add_constr(
        mip.xsum(nutrient_values_ex3[j][i] * x3[j] for j in foods_ex3) >= min_requirements[i],
        name=f"Nutrient_{i}"
    )

# --- 求解 ---
status3 = m3.optimize()

# --- 結果と考察 ---
print("--- 演習問題3 解答 ---")
if status3 == mip.OptimizationStatus.OPTIMAL:
    print(f"最小コスト: {m3.objective_value:.2f}円")
    print("\n[各食品の最適摂取量]")
    for j in foods_ex3:
        if x3[j].x > 1e-6:
            print(f"{j}: {x3[j].x:.2f} {food_units_ex3[j]}")
else:
    print("最適解が見つかりませんでした。")

--- 演習問題3 解答 ---
最小コスト: 147.73円

[各食品の最適摂取量]
卵: 1.14 1個
ヨーグルト: 4.55 100g


---

## 演習問題 4

### 1. 問題の確認
元の問題設定に加えて、「牛乳」の摂取量を最大2単位(200ml) までとする制約を追加した場合の最小コストを解答せよ。

### 2. 数理モデルの定式化
この問題では、特定の変数に対する上限制約が追加される。牛乳の摂取量を表す変数 $x_{牛乳}$ に対して、以下の制約をモデルに加える。

$$x_{牛乳} \le 2$$


### 3. Python (MIP) による実装
`m.add_constr()` を用いて、$x_{牛乳} \le 2$ という制約を明示的に追加する。あるいは、変数を定義する際に
`x1['牛乳'].ub=2.0`
と設定することでも実現できる。

In [7]:
# --- 演習問題4 ---

# --- モデルの作成 ---
m4 = mip.Model(name="Diet_Ex4", sense=mip.MINIMIZE)

# --- 変数の定義 ---
x4 = {j: m4.add_var(name=f"x_{j}", var_type=mip.CONTINUOUS, lb=0.0) for j in foods}

# --- 目的関数の設定 ---
m4.objective = mip.xsum(costs[j] * x4[j] for j in foods)

# --- 制約条件の追加 ---
# 栄養素の最小摂取量制約
for i in nutrients:
    m4.add_constr(
        mip.xsum(nutrient_values[j][i] * x4[j] for j in foods) >= min_requirements[i],
        name=f"Nutrient_{i}"
    )
# 牛乳の最大摂取量制約
m4.add_constr(x4['牛乳'] <= 2, name="Milk_upper_bound")

# --- 求解 ---
status4 = m4.optimize()

# --- 結果と考察 ---
print("--- 演習問題4 解答 ---")
if status4 == mip.OptimizationStatus.OPTIMAL:
    print(f"最小コスト: {m4.objective_value:.2f}円")
    print("\n[各食品の最適摂取量]")
    for j in foods:
        if x4[j].x > 1e-6:
            print(f"{j}: {x4[j].x:.2f} {food_df.loc[j, '単位']}")
else:
    print("最適解が見つかりませんでした。")

--- 演習問題4 解答 ---
最小コスト: 188.52円

[各食品の最適摂取量]
牛乳: 2.00 100ml
卵: 1.38 1個
パン: 1.75 1枚
チーズ: 0.74 20g


---

## 演習問題 5

### 1. 問題の確認
元の問題設定に加えて、以下の3つの条件を追加した場合の最小コストを解答せよ。
1.  総摂取カロリーは800 kcal以下とする。
2.  総摂取カルシウムは1000mg以下とする。
3.  牛乳とチーズから摂取する総カロリーが、全摂取カロリーの50%以下になるようにする。

### 2. 数理モデルの定式化
この問題では、複数の新しい制約が追加される。

1.  **カロリーの上限制約:**
$$\sum_{j \in F} a_{カロリー, j} x_j \le 800$$

2.  **カルシウムの上限制約:**
$$\sum_{j \in F} a_{カルシウム, j} x_j \le 1000$$

3.  **乳製品からのカロリー比率制約:**
牛乳とチーズの集合を $F_{dairy} = \{牛乳, チーズ\}$ とする。
$$\frac{\sum_{j \in F_{dairy}} a_{カロリー, j} x_j}{\sum_{j \in F} a_{カロリー, j} x_j} \le 0.5$$
この非線形制約は、分母を払うことで線形化できる。
$$\sum_{j \in F_{dairy}} a_{カロリー, j} x_j \le 0.5 \times \sum_{j \in F} a_{カロリー, j} x_j$$
これを整理すると、以下の線形制約となる。
$$(a_{カロリー, 牛乳} x_{牛乳} + a_{カロリー, チーズ} x_{チーズ}) - 0.5 \times \sum_{j \in F} a_{カロリー, j} x_j \le 0$$

これらの制約を元のモデルに追加する。

### 3. Python (MIP) による実装
栄養素の上限制約と、線形化された比率制約を `m.add_constr()` でモデルに追加する。

In [8]:
# --- 演習問題5 ---


# --- モデルの作成 ---
m5 = mip.Model(name="Diet_Ex5", sense=mip.MINIMIZE)

# --- 変数の定義 ---
x5 = {j: m5.add_var(name=f"x_{j}", var_type=mip.CONTINUOUS, lb=0.0) for j in foods}

# --- 目的関数の設定 ---
m5.objective = mip.xsum(costs[j] * x5[j] for j in foods)

# --- 制約条件の追加 ---
# 1. 栄養素の最小摂取量制約
for i in nutrients:
    m5.add_constr(
        mip.xsum(nutrient_values[j][i] * x5[j] for j in foods) >= min_requirements[i],
        name=f"Nutrient_min_{i}"
    )

# 2. カロリーとカルシウムの上限制約
m5.add_constr(mip.xsum(nutrient_values[j]['カロリー'] * x5[j] for j in foods) <= 800, name="Calorie_upper_bound")
m5.add_constr(mip.xsum(nutrient_values[j]['カルシウム'] * x5[j] for j in foods) <= 1000, name="Calcium_upper_bound")

# 3. 乳製品からのカロリー比率制約
dairy_products = ['牛乳', 'チーズ']
calories_from_dairy = mip.xsum(nutrient_values[j]['カロリー'] * x5[j] for j in dairy_products)
total_calories = mip.xsum(nutrient_values[j]['カロリー'] * x5[j] for j in foods)
m5.add_constr(calories_from_dairy <= 0.5 * total_calories, name="Dairy_calorie_ratio")

# --- 求解 ---
status5 = m5.optimize()

# --- 結果と考察 ---
print("--- 演習問題5 解答 ---")
if status5 == mip.OptimizationStatus.OPTIMAL:
    print(f"最小コスト: {m5.objective_value:.2f}円")
    print("\n[各食品の最適摂取量]")
    for j in foods:
        if x5[j].x > 1e-6:
            print(f"{j}: {x5[j].x:.2f} {food_df.loc[j, '単位']}")
else:
    print("最適解が見つかりませんでした。")
    if status5 == mip.OptimizationStatus.INFEASIBLE:
      print("実行不可能: 与えられた全ての制約を同時に満たす解は存在しません。")

--- 演習問題5 解答 ---
最小コスト: 170.83円

[各食品の最適摂取量]
牛乳: 4.17 100ml
卵: 1.25 1個
パン: 1.25 1枚
